In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

"""
TestXpecgen.py: Tests for  the `xpecgen` package.
"""

import unittest
import math

import fastcat as fc

# s = fc.calculate_spectrum(100, 12, 3, 10, epsrel=0.5, monitor=None)

In [3]:
class XpecgenTest(unittest.TestCase):
#     def test_clone(self):
#         """Test the Spectrum clone method"""
#         s2 = s.clone()
#         # Check same values
#         self.assertEqual(list(s.x), list(s2.x))
#         self.assertEqual(list(s.y), list(s2.y))
#         self.assertEqual(list(s.discrete), list(s2.discrete))
#         # Check alteration does not alter both instances
#         s.x[0] = 10.0
#         s.y[0] = 10.0
#         s.discrete[0][0] = 10.0
#         self.assertNotEqual(s.x[0], s2.x[0])
#         self.assertNotEqual(s.y[0], s2.y[0])
#         self.assertNotEqual(s.discrete[0][0], s2.discrete[0][0])

    def test_get_continuous_function(self):
        """Test the get_continuous_function method"""
        f = s.get_continuous_function()
        for p in zip(s.x, s.y):
            self.assertAlmostEqual(p[1], f(p[0]))

    def test_norm_functions(self):
        """Test the get_norm and set_norm methods"""
        s2 = s.clone()
        for w in [lambda x: 1, lambda x: x, fc.get_fluence_to_dose()]:
            s2.set_norm(13.0, w)
            self.assertAlmostEqual(s2.get_norm(w), 13.0)

    def test_attenuate(self):
        """Test the attenuate method"""
        # Unit distance, unit energy-independent attenuation coefficient -> attenuation with a factor 1/e
        s2 = s.clone()
        s2.attenuate(1, lambda x: 1)
        # Check continuous component changed
        for p in zip(s.y, s2.y):
            self.assertAlmostEqual(p[0] / math.e, p[1])
        # Check discrete component change
        for p in zip(s.discrete, s2.discrete):
            self.assertAlmostEqual(p[0][1] / math.e, p[1][1])

    def test_HVL_values(self):
        """Test to reproduce the values in Table III of Med. Phys. 43, 4655 (2016)"""

        # Calculate the emission spectra
        num_div = 20  # Points in each spectrum (quick calculation)
        s50 = fc.calculate_spectrum(50, 12, 3, num_div, epsrel=0.5, monitor=None)
        s80 = fc.calculate_spectrum(80, 12, 3, num_div, epsrel=0.5, monitor=None)
        s100 = fc.calculate_spectrum(100, 12, 3, num_div, epsrel=0.5, monitor=None)

        # Attenuate them
        s50.attenuate(0.12, fc.get_mu(13))  # 1.2 mm of Al
        s50.attenuate(100, fc.get_mu("air"))  # 100 cm of Air
        s80.attenuate(0.12, fc.get_mu(13))  # 1.2 mm of Al
        s80.attenuate(100, fc.get_mu("air"))  # 100 cm of Air
        s100.attenuate(0.12, fc.get_mu(13))  # 1.2 mm of Al
        s100.attenuate(100, fc.get_mu("air"))  # 100 cm of Air

        # Functions to calculate HVL in the sense of dose in Al
        fluence_to_dose = fc.get_fluence_to_dose()
        mu_al = fc.get_mu(13)

        # HVL in mm
        hvl50 = 10 * s50.hvl(0.5, fluence_to_dose, mu_al)
        hvl80 = 10 * s80.hvl(0.5, fluence_to_dose, mu_al)
        hvl100 = 10 * s100.hvl(0.5, fluence_to_dose, mu_al)

        self.assertAlmostEqual(hvl100, 2.37, places=1)
        self.assertAlmostEqual(hvl80, 1.85, places=1)
        self.assertAlmostEqual(hvl50, 1.20, places=1)

    def test_linear_combination(self):
        """Test linear combinations with Spectrum instances"""
        s2 = s.clone()
        s3 = s2+s2
        s4 = s2*2
        # Check same values
        self.assertEqual(list(s3.x), list(s4.x))
        self.assertEqual(list(s3.y), list(s4.y))
        self.assertEqual(list(s3.discrete), list(s4.discrete))

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.EEEE
ERROR: test_attenuate (__main__.XpecgenTest)
Test the attenuate method
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-3-98d554e8efac>", line 33, in test_attenuate
    s2 = s.clone()
NameError: name 's' is not defined

ERROR: test_get_continuous_function (__main__.XpecgenTest)
Test the get_continuous_function method
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-3-98d554e8efac>", line 19, in test_get_continuous_function
    f = s.get_continuous_function()
NameError: name 's' is not defined

ERROR: test_linear_combination (__main__.XpecgenTest)
Test linear combinations with Spectrum instances
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-3-98d554e8efac>", line 74, in test_linear_combination
    s2 = s.clone()
NameError: name 's' is not def

In [7]:
import unittest
import math
import os

import fastcat as fc

import numpy as np

class FastcatTest(unittest.TestCase):
    
    def test_dose_al(self):
        """
        Test if the dose matches MC estimates
        Testing the C 2.5 and W 2.5 MV
        """
        # Intialize
        angles = np.linspace(0,2*np.pi,2)
        s_fc = fc.Spectrum()
        phantom = fc.Catphan_515()
        # Test the C_25
        s_fc.load('C_spectrum_25')
        kernel = fc.Detector(s_fc, 'CuGOS-784-micrometer')
        dose = phantom.return_projs(kernel,s_fc,angles,nphoton=2e7,
                                    mgy=0,return_dose=True,verbose=0)
        
#         print(dose[-1])
        # C_25 test
        self.assertAlmostEqual(dose[-1], 0.04099205, places=2)
        
        # Test the W_25
        s_fc.load('W_spectrum_6')
        kernel = fc.Detector(s_fc, 'CuGOS-784-micrometer')
        dose = phantom.return_projs(kernel,s_fc,angles,nphoton=2e7,
                                    mgy=0,return_dose=True,verbose=0)
        # C_25 test
        self.assertAlmostEqual(dose[-1], 0.1935003, places=2)
        
    def test_counts_comparison(self):
        
        '''
        Compares the counts in a profile for the 515 phantom between MC and Fastcat
        3 Tests for the Al W and a kV spectrum
        '''
        
        s2 = fc.calculate_spectrum(120, 12, 3, 50,monitor=None)
        s2.attenuate(0.1,fc.get_mu(z=13))

        spectra = ['Al_spectrum_6',
                   'W_spectrum_6',
                   'kV']
        
        data_files = ['MC_MV_6Al_counts',
                     'MC_MV_6xW_counts',
                     'MC_kV_120_counts']
        
        angles = np.linspace(0,np.pi*2,2)        
        phantom = fc.Catphan_515()
        s = fc.Spectrum()

        for ii,spectrum in enumerate(spectra):

            if spectrum == 'kV':
                s = s2
            else:
                s.load(spectrum)

            kernel = fc.Detector(s, 'CuGOS-784-micrometer')
            counts = np.mean(phantom.return_projs(kernel,s,angles,det_on=False,mgy = 0.0)[0],0)
            MC_counts = np.load(os.path.join('test_data',data_files[ii] + '.npy'))
            self.assertLess(np.abs(np.max(counts - MC_counts)),10) 

    def test_MTF(self):
        
        '''
        See that the MTF compares to Shi et al. and Star Lack et al.
        Within four percent of shi and the star lack is far out now.
        '''
        s = fc.Spectrum()
        s.load('W_spectrum_6')#Varian_truebeam')
        kernel = fc.Detector(s, 'CuGOS-336-micrometer')
        kernel.get_plot_mtf_real(None)
        
        shi_x = np.load(os.path.join('test_data','shi_frequencies.npy'))
        shi_y = np.load(os.path.join('test_data','shi_MTF.npy'))
        
        int_mtf = np.interp(shi_x,kernel.freq,kernel.mtf)
        
#         print(np.max(np.abs(int_mtf - shi_y)))
        # Within 10 percent
        self.assertLess(np.max(np.abs(int_mtf - shi_y/shi_y[0])),0.04)
        
        # Star lack
        star_x = np.load(os.path.join('test_data','star_frequencies.npy'))
        star_y = np.load(os.path.join('test_data','star_MTF.npy'))
        
        kernel2 = fc.Detector(s, 'CWO-784-micrometer')
        kernel2.get_plot_mtf_real(None)

        int_mtf2 = np.interp(star_x,kernel2.freq,kernel2.mtf)
#         print(np.max(np.abs(int_mtf2 - star_y/star_y[0])))
        # Within 10 percent
        self.assertLess(np.max(np.abs(int_mtf2 - star_y/star_y[0])),0.20)
        
    def test_no_residual(self):
        '''
        Making sure that the scan profile is flat without an object
        And that the phantom reproduces the attenuation for water
        '''
        spectra = ['Varian_truebeam_phasespace']
        MV_detectors = ['CuGOS-784-micrometer']
        angles = np.linspace(np.pi/2,np.pi*2,2)      
        phantom = fc.Catphan_404()
        phantom.phan_map = ['1']*20
        s = fc.Spectrum()

        s.load(spectra[0])
        s.x = np.array([290,300,310,3000])
        s.y = np.array([0,1,0,0])
        kernel = fc.Detector(s, MV_detectors[0])
        kernel.deposition_interpolated = np.array([1,0,0])
        phantom.return_projs(kernel,s,angles,mgy=0.,scat_on=False,convolve_on=False)
        
        # Check that the profile is flat without object
        self.assertLess(np.mean(phantom.proj[0]),0.01)
        
        #Check that the correct attenuation is returned
        phantom.phan_map = ['water']*20
        
        phantom.return_projs(kernel,s,angles,mgy=0.,scat_on=False,convolve_on=False)
        self.assertAlmostEqual(np.max(phantom.proj[0]),0.97*0.1186*20*10,delta=0.1)
    def test_kV_experimental_profile(self):
        '''
        Test that the results match an experimental profile from Varian Truebeam within
        0.1, this is some error from geometrical distortion in the exp image
        '''
        s = fc.calculate_spectrum(100, 14, 5, 200,monitor=None)
        MV_detectors = ['CsI-784-micrometer']
        angles = np.linspace(np.pi/2,np.pi*2,2)      
        phantom = fc.Catphan_404()
        phantom.geomet.DSD = 1530
        kernel = fc.Detector(s, MV_detectors[0])
        kernel.add_focal_spot(1.2)
        phantom.phan_map = ['air','G4_POLYSTYRENE','G4_POLYVINYL_BUTYRAL','G4_POLYVINYL_BUTYRAL','CATPHAN_Delrin','G4_POLYVINYL_BUTYRAL','CATPHAN_Teflon_revised','air','CATPHAN_PMP','G4_POLYVINYL_BUTYRAL','CATPHAN_LDPE','G4_POLYVINYL_BUTYRAL','CATPHAN_Polystyrene','air','CATPHAN_Acrylic','air','CATPHAN_Teflon_revised','air','air','air','air'] 
        phantom.return_projs(kernel,s,angles,mgy=18,bowtie=True,filter='bowtie_asym2')
        
        exp_profile = np.load(os.path.join('test_data','kV_experimental_profile.npy'))
        
        self.assertLess(np.mean(np.abs(np.roll(phantom.proj[0,5],-3)-exp_profile)),1)
    
    def test_MV_experimental_profile(self):
        '''
        Test that the results match an experimental profile from Varian Truebeam within
        0.1, This one for the MV image.
        '''
        spectra = ['Varian_truebeam']
        MV_detectors = ['CuGOS-784-micrometer']
        angles = np.linspace(np.pi/2,np.pi*2,2)      
        phantom = fc.Catphan_404()
        phantom.geomet.DSD = 1510
        phantom.phan_map = ['air','G4_POLYSTYRENE','G4_POLYVINYL_BUTYRAL','G4_POLYVINYL_BUTYRAL','CATPHAN_Delrin','G4_POLYVINYL_BUTYRAL','CATPHAN_Teflon_revised','air','CATPHAN_PMP','G4_POLYVINYL_BUTYRAL','CATPHAN_LDPE','G4_POLYVINYL_BUTYRAL','CATPHAN_Polystyrene','air','CATPHAN_Acrylic','air','CATPHAN_Teflon_revised','air','air','air','air'] 
        s = fc.Spectrum()
        s.load(spectra[0])
        s.x[0] = 1
        s.x[1] = 2
        s.attenuate(0.,fc.get_mu(z=13)) #3.7
        kernel = fc.Detector(s, MV_detectors[0])
        kernel.add_focal_spot(1.4)
        phantom.return_projs(kernel,s,angles,mgy = 0.0,bowtie=True,filter='FF_t0')
        exp_profile = np.load(os.path.join('test_data','MV_experimental_profile.npy'))
        self.assertLess(np.mean(np.abs(np.roll(phantom.proj[0,5],-3)-exp_profile)),1)




if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.

I got here
I got here


.....
----------------------------------------------------------------------
Ran 6 tests in 45.606s

OK


In [27]:
%pylab widget

plt.figure()
shi_x = np.load(os.path.join('test_data','shi_frequencies.npy'))
shi_y = np.load(os.path.join('test_data','shi_MTF.npy'))

# Star lack
star_x = np.load(os.path.join('test_data','star_frequencies.npy'))
star_y = np.load(os.path.join('test_data','star_MTF.npy'))

s = fc.Spectrum()
s.load('W_spectrum_6')
kernel = fc.Kernel(s, 'CWO-784-micrometer')
kernel.get_plot_mtf_real(None)

int_mtf2 = np.interp(star_x,kernel.freq,kernel.mtf)
# plt.plot(shi_x,shi_y,star_x,star_y)
plt.plot(star_x,int_mtf2)
plt.plot(star_x,star_y)

print(np.max(np.abs(int_mtf2 - star_y/star_y[0])))

[2021-05-10 18:09:17,800] {pyplot.py:290} DEBUG - Loaded backend module://ipympl.backend_nbagg version unknown.
Populating the interactive namespace from numpy and matplotlib


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

I got here
0.7394813567717956
